This script extracts raw in-scanner output files from NAS (QLAB server) and populates the events.tsv files within the BIDS structure. This script was written by Owen Li.

Load all libraries

In [ ]:
import pandas as pd
from pandas import ExcelWriter
from openpyxl import load_workbook
import csv
import os

Establish folders and directories

In [ ]:
folder_name = 'sl_raw_data'               #Old folder name

auditory_folder = 'auditory_folder'
visual_folder = 'visual_folder'
auditory_times = pd.read_excel("EV_timing.xlsx", 'Auditory_Times', index = False)
visual_times = pd.read_excel("EV_timing.xlsx", 'Visual_Times', index = False)

auditory_times = auditory_times.drop(auditory_times.columns[2],axis=1)
visual_times = visual_times.drop(visual_times.columns[2],axis=1)

os.makedirs(auditory_folder) #Creat a new folder to save new auditory excel files
os.makedirs(visual_folder)  #Creat a new folder to save new visual excel files
auditory_files = []
visual_files = []

Run the following script to organize data

In [ ]:
for i in os.listdir(folder_name): #load every file from the folder
    if i.endswith('auditory_1.csv') or i.endswith('auditory_2.csv') or i.endswith('auditory_3.csv') or i.endswith('auditory_4.csv'):
        auditory_files.append(i)
    elif i.endswith('visual_1.csv') or i.endswith('visual_2.csv') or i.endswith('visual_3.csv') or i.endswith('visual_4.csv'):
        visual_files.append(i)


def save_new_folder(files, new_folder_name):
    for list in files:

        file = open(os.path.join(folder_name, list))  # load csv into the python
        numline = len(file.readlines())

        def read_cell(x, y):  # function that can read the cell
            with open(os.path.join(folder_name, list), 'r') as f:
                reader = csv.reader(f)
                y_count = 0
                for n in reader:
                    if y_count == y:
                        cell = n[x]
                        return cell
                    y_count += 1

        a = []  # creat a list to store R S B
        for i in range(1, numline + 1):
            a.append(read_cell(5, i))
        # cell = sheet.cell(row, 2)
        # for list in cell.value:
        #     a.append(list)

        inRow = []  # create another list to store R and S without repeat
        last = None
        for x in a:
            if last != x:
                inRow.append(x)
            last = x

        inRow = [x for x in inRow if x != 'B']  # Delete all the B's
        inRow = [x for x in inRow if x != None]
        print(inRow)

        s = 1
        num = []

        for y in inRow:
            if y == 'S':
                num.append(s)
                s += 1
            elif y == 'R':
                num.append(1)
        print(num)
        # for f in os.listdir(folder_name): #rename the new excel file which is .xlsx file
        f_name, f_ext = os.path.splitext(list)
        new_ext = '.xlsx'
        new_name = '{}{}'.format(f_name, new_ext)

        df = pd.DataFrame(inRow)  # write the result into a new excel
        dt = pd.DataFrame(num)
        writer = ExcelWriter(os.path.join(new_folder_name, new_name))  # save every file to new folder
        df.to_excel(writer, 'Sheet1', index=False, header=None, startrow=0, startcol=2)
        dt.to_excel(writer,'Sheet1',index=False, header=None, startrow=0, startcol=3)
        writer.save()


save_new_folder(auditory_files, auditory_folder)
save_new_folder(visual_files,visual_folder)

def save_time(folder, time):
    for i in os.listdir(folder):
        book = load_workbook(os.path.join(folder, i))
        writer = pd.ExcelWriter(os.path.join(folder, i), engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        time.to_excel(writer, 'Sheet1', index=False, startrow=0, startcol=0)
        writer.save()

save_time(auditory_folder,auditory_times)
save_time(visual_folder,visual_times)




for i in os.listdir('auditory_folder'):
    for x in range(len(os.listdir('auditory_folder'))):
        for y in range(4):
            if ('blast_a_{}_auditory_{}.xlsx'.format('{0:03}'.format(x+1), y+1)) == i:
                time = pd.read_excel('auditory_folder/blast_a_{}_auditory_{}.xlsx'.format('{0:03}'.format(x+1), y+1), index=False)
                with open(os.path.join('/Volumes/data/projects/blast/data/bids/sub-blasta{}/func/sub-blasta{}_task-asl_run-{}_events.tsv'.format('{0:03}'.format(x+1),'{0:03}'.format(x+1),
                                                                                                          '{0:02}'.format(y+1))), 'wt') as out_file:
                    tsv_writer = csv.writer(out_file, delimiter='\t')
                    tsv_writer.writerow(['onset', 'duration', 'trial_type', 'block'])
                    time.to_csv(out_file, index=False, sep='\t')
                print(('blast_a_{}_auditory_{}.xlsx'.format('{0:03}'.format(x+1), y+1)))


for i in os.listdir('visual_folder'):
    for x in range(len(os.listdir('visual_folder'))):
        for y in range(4):
            if ('blast_a_{}_visual_{}.xlsx'.format('{0:03}'.format(x+1), y+1)) == i:
                time = pd.read_excel('visual_folder/blast_a_{}_visual_{}.xlsx'.format('{0:03}'.format(x+1), y+1), index=False)
                with open(os.path.join('/Volumes/data/projects/blast/data/bids/sub-blasta{}/func/sub-blasta{}_task-vsl_run-{}_events.tsv'.format('{0:03}'.format(x+1),'{0:03}'.format(x+1),
                                                                                                          '{0:02}'.format(y+1))), 'wt') as out_file:
                    tsv_writer = csv.writer(out_file, delimiter='\t')
                    tsv_writer.writerow(['onset', 'duration', 'trial_type', 'block'])
                    time.to_csv(out_file, index=False, sep='\t')
                print(('blast_a_{}_visual_{}.xlsx'.format('{0:03}'.format(x+1), y+1)))